In [1]:
from modeling.compute import Var
from modeling.execution import Component
from foundations.functionalsets import FunctionalComp
from foundations.functionalsets import ResidualSet
from modeling.api import create_component
from modeling.couplingapi import Subproblem
from sympy import exp

In [2]:
z1,z2,x,y2 = Var('z_1'), Var('z_2'), Var('x'), Var('y_2')
a, y1 = Var('a'), Var('y_1')

In [3]:
F1 = FunctionalComp(Component.fromsympy(
    z2+x-0.2*y2, a, arg_mapping=True))
F2 = FunctionalComp(Component.fromsympy(
    z1**2+a, y1, arg_mapping=True))
F3 = FunctionalComp(Component.fromsympy(
    y1**0.5+z1+z2, y2, arg_mapping=True))
obj = Component.fromsympy(
    x**2+z2+y1+exp(-y2), arg_mapping=True)
ineq1 = Component.fromsympy(
    3.16-y1, arg_mapping=True)
ineq2 = Component.fromsympy(
    y2-24, arg_mapping=True)

In [4]:
functionals = [F1, F2, F3]
S = ResidualSet(sum([functional.projectable.components 
                     for functional in functionals], []))
independents = sum([functional.independent for functional in functionals],())
projectto = tuple(set(elt for functional in functionals 
               for elt in functional.projected if elt not in independents))

In [5]:
y = {x:1, z2:1, z1:1}

In [6]:
F = S.project(projectto)
sol = F.solve(y)

In [7]:
J = lambda y: obj.evaldict({**y, **F.solve(y)})

In [8]:
J({x:4, z2:2, z1:-1})

DeviceArray(24.32798941, dtype=float64)

In [9]:
z = {**y, **sol}

In [10]:
F1.solve(z), F2.solve(z), F3.solve(z)

({a: DeviceArray(1.29689011, dtype=float64)},
 {y_1: DeviceArray(2.29689011, dtype=float64)},
 {y_2: DeviceArray(3.51554944, dtype=float64)})

In [11]:
obj.evaldict(z), ineq1.evaldict(z), ineq2.evaldict(z)

(DeviceArray(4.32662157, dtype=float64),
 0.8631098884280703,
 -20.48445055785965)